# imports

In [1]:
from mylib import base, dataset
from mylib.model_classification import ModelClass
from mylib.model_base import ModelBase

from dotenv import dotenv_values
from pathlib import Path
from pprint import pprint, pformat

import pandas as pd
import pandas.api.types as pd_types

import numpy as np

import plotly.express as plotly_px

import joblib

import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 50) # Устанавливаем максимальное количество отображаемых столбцов равным 50
#pd.set_option('display.max_rows', 50) # Устанавливаем максимальное количество отображаемых строк равным 20
pd.options.display.float_format = '{:.5f}'.format # Устанавливаем формат отображения чисел с двумя знаками после запятой
pd.options.mode.use_inf_as_na = True # Настройка режима Pandas для рассмотрения бесконечностей (inf) как пропущенных значений (NA)

# Конфигурация формата отображения графиков в виде векторных изображений
%config InlineBackend.figure_format = 'svg'

# для построения графиков внутри Jupyter Notebook
%matplotlib inline

# 1. Загрузка конфигов

In [3]:
# загрузить параметры
settings_dict = {
    **dotenv_values("settings")
}

settings = base.Settings(settings_dict)
settings.enviroment["RANDOM_STATE"] = int(settings.enviroment["RANDOM_STATE"])
n_jobs = -1
verbose = 1

In [4]:
pprint(settings)

Settings(enviroment={'CACHE_SUBFOLDER': 'cached_results',
                     'DATASET_FILENAME_AFTER_EDA': '01_dataset_df_after_EDA.joblib',
                     'DATASET_FILENAME_READY_FOR_TRAIN_ORIGINAL': '02_dataset_catencoder_df_ready_for_train_original.joblib',
                     'DATASET_SUBFOLDER': 'dataset',
                     'GRID_SEARCH_TEMPLATE_FILENAME': '03_GridSearch_%s.pickle',
                     'MODEL_CLASS_TEMPLATE_FILENAME': '04_model_%s.pickle',
                     'PARAMS_FILENAME_AFTER_EDA': '01_params_after_EDA.joblib',
                     'PARAMS_FILENAME_READY_FOR_TRAIN_ORIGINAL': '02_params_ready_for_train_original.joblib',
                     'RANDOM_STATE': 42,
                     'RESULT_SUBFOLDER': 'result'})


# Загрузитить датасет и подготовить его для обучения моделей

In [5]:
dataset_df = joblib.load(Path(settings.result_folder, settings.enviroment["DATASET_FILENAME_READY_FOR_TRAIN_ORIGINAL"]))
paams = joblib.load(Path(settings.result_folder, settings.enviroment["PARAMS_FILENAME_READY_FOR_TRAIN_ORIGINAL"]))
#dataset_df = dataset_df.sample(100, random_state=42)
X_train, X_test, y_train, y_test = dataset.prepare_dataset(dataset_df, paams)

In [6]:
np.mean(y_train), np.mean(y_test)

(0.36726766344180595, 0.36730506155950754)

In [46]:
y_train.nunique()

elements, repeats = np.unique(y_train, return_counts=True)
index = repeats.argmax()
elem0 = elements[0]
elem1 = elements[1]

repeats/repeats.sum()

array([0.63273234, 0.36726766])

# Модели классификации

## RandomForest Classificator

In [7]:
# Decision Tree Classifier
from sklearn.tree import (
    DecisionTreeClassifier,  # Классификатор на основе дерева решений
    plot_tree,  # Функция для визуализации дерева решений
    export_text,  # Функция для экспорта дерева решений в текстовом формате
    export_graphviz  # Функция для экспорта дерева решений в формат Graphviz
)

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

# Импорт функций для визуализации дерева решений
from sklearn.tree import export_graphviz  # Функция для экспорта дерева решений в формат Graphviz
#import graphviz  # Библиотека для визуализации графов
from mlxtend.plotting import plot_decision_regions  # Функция для визуализации границ решений
import tqdm
from sklearn.model_selection import train_test_split, cross_val_score

rfc_model_name = 'RF_C'

## !!!!!!!!!!!!!

In [33]:
rfc_grid_search_accuracy = ModelBase.load_or_create_and_fit_GridSearchCV(rfc_model_name+"_accuracy_1", RandomForestClassifier, 
                                                                         {'random_state': [42],
                                                                          'n_estimators': [x for x in range(10, 100, 20)],
                                                                          'criterion': ['gini', 'entropy', 'log_loss'],
                                                                         }, 
                                                                         X_train, y_train, settings,
                                                                         n_jobs=n_jobs, verbose=verbose,
                                                                         scoring='f1')
pprint(rfc_grid_search_accuracy.best_params_)
print(rfc_grid_search_accuracy.best_score_)       

Создается и выполняется GridSearchCV для модели RF_C_accuracy_1 класса RandomForestClassifier
Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'criterion': 'gini', 'n_estimators': 90, 'random_state': 42}
0.7067945405222263


In [34]:
rfc_grid_search_accuracy = ModelBase.load_or_create_and_fit_GridSearchCV(rfc_model_name+"_accuracy_2", RandomForestClassifier, 
                                                                         {'random_state': [42],
                                                                          'n_estimators': [90],
                                                                          'criterion': ['gini'],
                                                                          'max_depth': [3, 6, 9],
                                                                          'min_samples_leaf': [3, 5, 7],
                                                                          'min_samples_split': [3, 5, 7],
                                                                          'max_features': ['sqrt', 'log2']
                                                                         }, 
                                                                         X_train, y_train, settings,
                                                                         n_jobs=n_jobs, verbose=verbose,
                                                                         scoring='f1')
pprint(rfc_grid_search_accuracy.best_params_)
print(rfc_grid_search_accuracy.best_score_)

Создается и выполняется GridSearchCV для модели RF_C_accuracy_2 класса RandomForestClassifier
Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 3,
 'n_estimators': 90,
 'random_state': 42}
0.6667281623583867


In [30]:
rfc_grid_search_accuracy = ModelBase.load_or_create_and_fit_GridSearchCV(rfc_model_name+"_accuracy_3", RandomForestClassifier, 
                                                                         {'random_state': [42],
                                                                          'n_estimators': [90],
                                                                          'criterion': ['gini'],
                                                                          'max_depth': [None],
                                                                          'max_features': ['sqrt'],
                                                                          'min_samples_leaf': [3],
                                                                          'min_samples_split': [3],
                                                                          'ccp_alpha': [0, 0.01, 0.25, 0.05, 0.1, 0.5, 1],
                                                                          #'max_samples': [0.4, 0.6, 0.8],
                                                                         }, 
                                                                         X_train, y_train, settings,
                                                                         n_jobs=n_jobs, verbose=verbose,
                                                                         scoring='accuracy')
pprint(rfc_grid_search_accuracy.best_params_)
print(rfc_grid_search_accuracy.best_score_)

Создается и выполняется GridSearchCV для модели RF_C_accuracy_3 класса RandomForestClassifier
Fitting 5 folds for each of 7 candidates, totalling 35 fits
{'ccp_alpha': 0,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 3,
 'n_estimators': 90,
 'random_state': 42}
0.7844475006401022


### Поиск оптимальных параметров модели

Сделаем поиск сравнивая модели по трём разным метриках - accuracy, f1, roc_auc

### Обучим модель по наилучшим подобранным параметрам

In [35]:
print("Подобранные гиперпараметры модели:")
pprint(rfc_grid_search_accuracy.best_params_)

Подобранные гиперпараметры модели:
{'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 3,
 'n_estimators': 90,
 'random_state': 42}


In [36]:
rfc_model = ModelClass.load_or_create_and_fit_model(rfc_model_name, RandomForestClassifier, 
                                                    rfc_grid_search_accuracy.best_params_, 
                                                    X_train, X_test, y_train, y_test,
                                                    settings)

rfc_model.show_quality()

Создается и тренируется модель RF_C класса RandomForestClassifier


Получившаяся модель непло